In [21]:
from google.colab import files
x=files.upload()

Saving main_new.csv to main_new.csv


In [0]:
def convert_to_int(word):
    word_dict = {'Coronavirus':0, 'AskIndia':1, 'Non-Political':2, 'Policy/Economy':3, 'Business/Finance':4, 'Science/Technology':5, 'Sports':6, 'Politics':7,
                'Food':8}
    return word_dict[word]
flair_list=['Coronavirus','AskIndia','Non-Political','Policy/Economy','Business/Finance','Science/Technology','Sports','Politics','Food']

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [0]:
import csv
labels=[]
sentences=[]
counts=[0,0,0,0,0,0,0,0,0]
with open("main_new.csv") as csvfile:
  reader=csv.reader(csvfile,delimiter=',')
  next(reader)
  for row in reader:
      if((row[1] in flair_list) and counts[flair_list.index(row[1])]<2000):
          counts[flair_list.index(row[1])]+=1
          labels.append(convert_to_int(row[1]))
          sentences.append(row[2])

In [0]:
import random
c = list(zip(sentences, labels))

random.shuffle(c)

sentences, labels = zip(*c)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(sentences,labels,random_state=0)

vect = CountVectorizer().fit(X_train)
X_train_vectorized = vect.transform(X_train)
X_test_vectorized = vect.transform(X_test)
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(y_train)
Test_Y = Encoder.fit_transform(y_test)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(sentences)
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [35]:
#Naive Bayes classifier
alpha=0.4800000000000003
clfrNB = MultinomialNB(alpha = alpha)
clfrNB.fit(X_train_vectorized, y_train)
preds = clfrNB.predict(vect.transform(X_test))
score = accuracy_score(y_test, preds)
print("Accuracy:", score*100, "%")


Accuracy: 65.07547617165889 %


In [36]:
#Random Forest Classifier

clf = RandomForestClassifier()
clf.fit(X_train_vectorized, y_train)
y_pred = clf.predict(X_test_vectorized)


print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 438   48  109    3    0    0    1   81    1]
 [  33 1110  389    9   25    4    2  146    3]
 [  96  369 2094   25   59   21   17  530    6]
 [  16   48  221  248   49    7    3  280    3]
 [   8   87  280   64  326   15    1   71    4]
 [  10   91  221   10   23  125    1   53    1]
 [   0   21   81    0    3    0  142   17    0]
 [  42  131  636   47   16    8   19 3476    3]
 [   2   30   53    0    4    0    0    4   33]]
              precision    recall  f1-score   support

           0       0.68      0.64      0.66       681
           1       0.57      0.64      0.61      1721
           2       0.51      0.65      0.57      3217
           3       0.61      0.28      0.39       875
           4       0.65      0.38      0.48       856
           5       0.69      0.23      0.35       535
           6       0.76      0.54      0.63       264
           7       0.75      0.79      0.77      4378
           8       0.61      0.26      0.37       126

    accuracy             

In [37]:
#SVM classifier
#12% accuracy when using polynomial kernels
SVM = svm.SVC(C=1.0, kernel='linear', gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

predictions_SVM = SVM.predict(Test_X_Tfidf)

print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  64.88579783450565


In [0]:
#Logistic Regression

logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(Train_X_Tfidf, y_train)
y_pred = logreg.predict(Test_X_Tfidf)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.5318877551020408
Testing F1 score: 0.5356255338415167


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
#MLP Classifier

classifier = MLPClassifier(hidden_layer_sizes=(50,150,50), max_iter=20,activation = 'relu',solver='adam',random_state=1)
classifier.fit(X_train_vectorized, y_train)

y_pred = classifier.predict(X_test_vectorized)

print('Testing accuracy %s' % accuracy_score(y_test, y_pred))


Testing accuracy 0.536734693877551


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
